In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import polars as pl
import numpy as np
from sklearn.preprocessing import RobustScaler

import pandas as pd

class FeedForwardNN(nn.Module):
    def __init__(self, input_size, hidden_size=32):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)
        self.activation = nn.Tanh()

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = self.fc3(x)
        return x

# rawData = pd.read_excel('../data/dundalk_2020.xlsx')
rawData = pl.read_excel('../data/dundalk.xlsx').to_pandas()


In [8]:
lstmData = rawData.copy()
lstmData.set_index('Timestamps',inplace=True)
lstmData = lstmData[['WindSpeed','Power']]
lstmData = lstmData.loc['2006-01-30':'2006-03-02']

In [9]:
mlp = FeedForwardNN(input_size=3)
mlp.load_state_dict(torch.load('../models/ffnn.pth'))

xScaler:RobustScaler = torch.load('../models/xscaler_ffnn.pth')
yScaler:RobustScaler = torch.load('../models/yscaler_ffnn.pth')

d:\Program Files\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning:

Trying to unpickle estimator RobustScaler from version 1.5.1 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations



In [ ]:
from raghav.WindSpeed.LSTM import WindPowerLSTM, create_sequences, getpredictions

In [11]:
model_path = 'raghav/WindSpeed/lstm_model2.pt'
scaler_x = 'raghav/WindSpeed/minmax_scaler_x2.pt'
scaler_y = 'raghav/WindSpeed/minmax_scaler_y2.pt'
sacler_y = torch.load(scaler_y,map_location=torch.device('cpu') if not torch.cuda.is_available() else None)
seq_length = 30
preds =  getpredictions(WindPowerLSTM, model_path, scaler_x, lstmData.drop(columns = 'Power'), seq_length=seq_length)
preds = sacler_y.inverse_transform(preds)

d:\Program Files\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning:

Trying to unpickle estimator MinMaxScaler from version 1.6.1 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations

d:\Program Files\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning:

Trying to unpickle estimator MinMaxScaler from version 1.6.1 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations



In [14]:
# lstmData['PredictedPower'] = preds
results=lstmData.copy()
results = results.iloc[seq_length:]
results['PredictedPower'] = preds
results.reset_index(drop=False,inplace=True)
(
    results.melt(id_vars = 'Timestamps',value_vars=['Power','PredictedPower'])
    .loc[lambda s: s.Timestamps >= '2006-01-30']
    .loc[lambda s: s.Timestamps < '2006-03-01']
    .plot(x = 'Timestamps', y = 'value', color = 'variable', backend = 'plotly', height = 600)
    .update_layout(title = 'Power vs Predicted Power - LSTM', xaxis_title = 'Timestamps', yaxis_title = 'Power')
)

In [19]:
(
    rawData[['Timestamps','Power','WindSpeed','WindDirAbs','EnvirTemp']]
    .loc[lambda s: s.Timestamps >= '2006-01-30']
    .loc[lambda s: s.Timestamps < '2006-03-01']    
    .assign(PredictedPower = lambda df: mlp(
        torch.FloatTensor(
            xScaler.transform(
                df[['WindSpeed','WindDirAbs','EnvirTemp']]
                )
            )
        ).detach().numpy()
    )
    .assign(PredictedPower = lambda s: yScaler.inverse_transform(s.PredictedPower.values.reshape(-1,1)))
    .melt(id_vars = 'Timestamps',value_vars=['Power','PredictedPower'])
    .plot(x = 'Timestamps', y = 'value', color = 'variable', backend = 'plotly', height = 600)
    .update_layout(title = 'Power vs Predicted Power - MLP', xaxis_title = 'Timestamps', yaxis_title = 'Power')
)